# 필리버스터 워드 클라우드

이 노트북에서는 2016년 테러방지법 반대 필리버스터의 속기록을 분석하여 키워드들을 워드 클라우드로 나타내고, TF-IDF 기법으로 국회의원들의 발언을 비교해서 의원 별로 특징을 시각화해보고자 한다.

### 목차

이 분석은 다음과 같이 다섯 단계로 나눌 수 있다.

1. 데이터 수집 및 전처리
2. 워드 클라우드 그리기
3. 의원 별로 발언 나눠 보기
4. 분석 대상 단어 선정
5. TF-IDF로 의원 별 발언 특징 잡아내기

## 데이터 수집 및 전처리

국회에서의 모든 토론은 [국회회의록 사이트](http://likms.assembly.go.kr/record/index.jsp)에서 속기록을 볼 수 있다. 2016년 테러방지법 반대 필리버스터의 정식명칭인 **제340회 국회(임시회) 제7차 본회의에 대한 무제한 토론**으로 검색하여 문서를 다운받을 수 있다. 문서 형식은 pdf 파일과 hwp 파일로 제공되는데, 여기서는 필리버스터 토론에 대한 속기록만 추출해서 따로 저장한 텍스트 파일을 바탕으로 분석을 진행하도록 하겠다.

In [1]:
filibuster_file = 'data/filibuster.txt'

with open(filibuster_file) as f:
    text = f.read()

2016년 2월 23일부터 3월 2일까지 192시간 27분 동안 이어진 필리버스터 속기록은 350만 자, 26만 줄에 이르는 장대한 기록이다. 심지어 속기록 전문이 1300페이지의 [책](http://www.aladin.co.kr/shop/wproduct.aspx?ItemId=79175416)으로 출간되기도 했다. 그 내용을 잠깐 살펴보자.

In [2]:
print(text[:1200])

◯김광진 의원  사랑하고 존경하는 국민 여러분! 
  국회의장님과 선배․동료 의원 여러분! 
  오늘 저는 테러방지법과 관련해서 무제한 토론을 신청했습니다. 
  저는 국회 정보위원회의 법안소위 위원입니다.
  대한민국의 어떤 누구도 대한민국의 안보와 그리고 테러를 막겠다라고 하는 것에 대해서 반대할 국민은 없을 것입니다. 국민뿐만 아니라 정치권 누구도 그것을 반대하지는 않을 것입니다. 
  그러나 대한민국은 민주주의 국가입니다. 국회라고 하는 공간을 통해서 민의를 반영하는 공간이지요. 그리고 그 국회는 상임위원회에서 관련한 논의들을 쭉 해 오고 있습니다. 
  국회 정보위원회는 테러방지법과 관련해서 네 차례에 걸친 회의를 하고 있는 중이었습니다. 그 과정에 있어서 잠깐 멈추기도 하고 또 이후에 다른 법률의 문제를 제기하기도 했고, 사실 19대 국회 기간 정보위원회의 법안소위는 단 한 차례도 열리지 않고 있었기 때문에 다른 법률과의 연계도, 그동안 쌓여 있었던 정보위원회의 법안들도 같이 논의하는 것이 좋겠다라고 하는 생각들도 많이 있었습니다. 그래서 그런 것들과 함께 논의하자라고 하는 것들에 대한 얘기도 있었고 또 하나는 국정원과 관련한 여러 가지 불신에 대한 문제, 지금 이 테러방지법의 논의가 가장 큰 것이 그것이겠습니다만 국정원에 대한 불신에 대한 문제를 해소하는 것이 여야를 떠나서 국민 모두에게 좋은 일이기 때문에 관련한 것들에 대한 법안도 같이 논의하는 것이 좋겠다라고 하는 논의를 하고 있는 중이었습니다. 그게 정상적인 국회의 운영 방식이라고 저는 생각합니다.
  저는 초선의 비례대표 의원입니다. 4년밖에 의정활동을 해 오지 않았습니다만 이 자리에 재선, 3선, 4선, 그보다 더 많은 선수를 하고 계신 선배․동료 의원 여러분도 계신 것을 압니다. 국회의 상임위에서 정상적인 절차를 거쳐서 논의하는 것, 그것이 민주주의의 기본이라고 생각합니다. 그것은 초선인 저도, 누구나 알 수 있는 것입니다. 
  그런데 오늘 국회가 열렸습니다. 언제나 합리적으로 평가받

국회 속기록은 맞춤법에 맞고 일정 규칙에 따라 정리되어 있기 때문에 분석이 용이하다. 그러나 이 텍스트 역시 분석 전에 전처리 과정을 거쳐야 한다. '은', '는', '이', '가' 같은 조사나 '있는', '것' 같은 자주 쓰이지만 별 의미 없는 단어들이 핵심어를 한 눈에 보기 어렵게 만들기 때문이다. 전처리를 통해 노이즈들을 제거한 후에야 의미 있는 워드 클라우드를 얻을 수 있다. 전처리 과정은 다음과 같이 진행했다.

1. konlpy 라이브러리의 Twitter 모듈로 명사만을 추출한다.
2. 길이가 한 글자인 단어는 제외한다.
3. 위 과정으로 제거하지 못하는 노이즈는 분석 과정에서 발견하여 수작업으로 stop words에 추가하여 제거한다.

In [3]:
from konlpy.tag import Twitter
tw = Twitter()
stop_words = open('intermediate/stop_words.txt').read().split('\n')

def normalize(text):
    nouns = tw.nouns(text)
    filtered_nouns = [ noun for noun in nouns if len(noun) > 1 and noun not in stop_words]
    return filtered_nouns

In [4]:
%%time
normalized_text = normalize(text)

CPU times: user 2min 9s, sys: 1.53 s, total: 2min 11s
Wall time: 1min 44s


In [5]:
with open('intermediate/filibuster_normalized.txt', 'w') as f:
    f.write(' '.join(normalized_text))

전처리 과정을 거치고 텍스트가 어떻게 바뀌었는지 살펴보자.

In [6]:
print(normalized_text[:200])

['김광진', '의원', '국민', '여러분', '국회의장', '선배', '동료', '의원', '여러분', '오늘', '테러', '방지법', '무제한', '토론', '국회', '정보', '위원회', '법안', '소위', '위원', '대한민국', '누구', '대한민국', '안보', '테러', '국민', '국민', '정치권', '누구', '그것', '대한민국', '민주주의', '국가', '국회', '공간', '공간', '이지', '국회', '상임', '위원회', '논의', '국회', '정보', '위원회', '테러', '방지법', '차례', '회의', '과정', '잠깐', '이후', '다른', '법률', '문제', '제기', '사실', '국회', '기간', '정보', '위원회', '법안', '소위', '차례', '때문', '다른', '법률', '그동안', '정보', '위원회', '법안', '생각', '대한', '얘기', '하나', '국정원', '여러', '가지', '불신', '대한', '문제', '지금', '테러', '방지법', '논의', '가장', '그것', '국정원', '대한', '불신', '대한', '문제', '여야', '국민', '모두', '이기', '때문', '대한', '법안', '논의', '그게', '정상', '국회', '운영', '방식', '초선', '비례대표', '의원', '의정', '활동', '오지', '자리', '재선', '선수', '선배', '동료', '의원', '여러분', '압니', '국회', '상임', '정상', '절차', '그것', '민주주의', '기본', '그것', '초선', '누구', '오늘', '국회', '합리', '평가', '정의화', '의장', '테러', '방지법', '심사', '기일', '오늘', '처음', '지정', '의장', '직권상정', '본회의', '부의', '박근혜', '대통령', '그동안', '일방', '통행', '불통', '입법부', '수장', '국회의장', '국회법', '천재지변', '전시', '사변', '국가', 

위에서 본 김광진 의원의 발언이 명사들의 리스트로 바뀌었다. 이 단어들이 앞으로 진행될 분석의 바탕이 된다.

## 워드 클라우드 그리기

앞에서 전처리한 텍스트를 워드 클라우드로 시각화하여 키워드를 찾아보도록 하자. 

워드 클라우드를 그리는 데는 [WordCloud 모듈](https://github.com/amueller/word_cloud)을 이용했다. 기본 설정에는 한글 폰트가 없으므로 한글을 나타내기 위해서는 한글 폰트를 직접 지정해주어야 한다.

In [7]:
from wordcloud import WordCloud
font_file = 'resources/Daum_SemiBold.ttf'
WC = WordCloud(font_path=font_file, width=700, height=700)

In [29]:
filibuster_wc = WC.generate(' '.join(normalized_text))
filibuster_wc.to_file('figures/filibuster.png')

![필리버스터 워드 클라우드](figures/filibuster.png)

워드 클라우드에서 다양한 키워드를 찾아볼 수 있다. 가장 큰 키워드는 **테러**이다. 이 필리버스터가 테러방지법을 반대하는 토론이었으므로 테러가 주제인 것은 당연할 것이다. 또한 이 워드 클라우드에서는 의원들이 테러방지법을 반대하는 이유도 살펴볼 수 있다. 그 이유는 **국정원** 때문이다. 필리버스터 전반적으로 의원들은 테러방지법이 악용되어 **국민**의 **정보**를 **국가**가 마음대로 수집하는 데 이용될 것이라는 불신을 드러내고 있다.

## 의원 별로 발언 나눠 보기

첫 주자인 김광진 의원부터 마지막 주자인 이종걸 의원까지 총 38명의 의원이 필리버스터 중에 토론에 나섰다. 필리버스터 전체의 키워드를 알아보는 것도 재미있지만, 각 의원들의 발언을 비교해서 그 차이를 알아보는 것도 의미가 있을 것이다. 본격적인 비교에 들어가기 전에, 먼저 필리버스터 전체 텍스트를 의원 별로 나누는 전처리 과정을 거치자.

필리버스터에서 발언한 의원들을 목록으로 만들었다. 국민의당 권은희 의원은 새누리당에 동명이인의 의원이 존재하여 속기록 상 權垠希로 표기되었다.

In [9]:
names = ['김광진', '문병호', '은수미', '박원석', '유승희', '최민희', '김제남', '신경민', '강기정', '김경협', 
         '서기호', '김현', '김용익', '배재정', '전순옥', '추미애', '정청래', '진선미', '최규성', '오제세', 
         '박혜자', '權垠希', '이학영', '홍종학', '서영교', '최원식', '홍익표', '이언주', '전정희', '임수경', 
         '안민석', '김기준', '김관영', '박영선', '주승용', '정진후', '심상정', '이종걸']

속기록에서는 새로운 사람이 발언을 시작하면 '◯' 기호 뒤에 그 이름을 써주는 규칙이 있다. 이 표기법을 이용해서 속기록에 등장하는 사람들의 발언을 따로 분리해내었다.

In [10]:
import re

by_name = {} # 의원의 이름을 key로, 그 발언을 value로 하는 dictionary
for line in text.split('\n'):
    line = line.strip()
    
    # 상황 설명 문장 제거
    if line.startswith('(') and line.endswith(')'): 
        continue
        
    if line.startswith('◯'): # 새로운 사람이 발언 시작
        current_name = re.findall(r'◯(\w+) ', line)[0] # 지금 발언하는 사람 추출
        line = line[line.find('  '):].strip()
    by_name.setdefault(current_name, []).append(line)

by_speaker_path = 'intermediate/by_speaker/'

for name in names:
    with open(by_speaker_path + name + '.txt', 'w') as f:
        f.write('\n'.join(by_name[name]))

첫번째 주자인 김광진 의원의 발언만을 살펴보면 다음과 같다.

In [11]:
by_name['김광진'][:9]

['사랑하고 존경하는 국민 여러분!',
 '국회의장님과 선배․동료 의원 여러분!',
 '오늘 저는 테러방지법과 관련해서 무제한 토론을 신청했습니다.',
 '저는 국회 정보위원회의 법안소위 위원입니다.',
 '대한민국의 어떤 누구도 대한민국의 안보와 그리고 테러를 막겠다라고 하는 것에 대해서 반대할 국민은 없을 것입니다. 국민뿐만 아니라 정치권 누구도 그것을 반대하지는 않을 것입니다.',
 '그러나 대한민국은 민주주의 국가입니다. 국회라고 하는 공간을 통해서 민의를 반영하는 공간이지요. 그리고 그 국회는 상임위원회에서 관련한 논의들을 쭉 해 오고 있습니다.',
 '국회 정보위원회는 테러방지법과 관련해서 네 차례에 걸친 회의를 하고 있는 중이었습니다. 그 과정에 있어서 잠깐 멈추기도 하고 또 이후에 다른 법률의 문제를 제기하기도 했고, 사실 19대 국회 기간 정보위원회의 법안소위는 단 한 차례도 열리지 않고 있었기 때문에 다른 법률과의 연계도, 그동안 쌓여 있었던 정보위원회의 법안들도 같이 논의하는 것이 좋겠다라고 하는 생각들도 많이 있었습니다. 그래서 그런 것들과 함께 논의하자라고 하는 것들에 대한 얘기도 있었고 또 하나는 국정원과 관련한 여러 가지 불신에 대한 문제, 지금 이 테러방지법의 논의가 가장 큰 것이 그것이겠습니다만 국정원에 대한 불신에 대한 문제를 해소하는 것이 여야를 떠나서 국민 모두에게 좋은 일이기 때문에 관련한 것들에 대한 법안도 같이 논의하는 것이 좋겠다라고 하는 논의를 하고 있는 중이었습니다. 그게 정상적인 국회의 운영 방식이라고 저는 생각합니다.',
 '저는 초선의 비례대표 의원입니다. 4년밖에 의정활동을 해 오지 않았습니다만 이 자리에 재선, 3선, 4선, 그보다 더 많은 선수를 하고 계신 선배․동료 의원 여러분도 계신 것을 압니다. 국회의 상임위에서 정상적인 절차를 거쳐서 논의하는 것, 그것이 민주주의의 기본이라고 생각합니다. 그것은 초선인 저도, 누구나 알 수 있는 것입니다.',
 '그런데 오늘 국회가 열렸습니다. 언제나 합리적으로 

마지막 주자인 이종걸 의원의 발언만을 살펴보면 다음과 같다.

In [12]:
by_name['이종걸'][:9]

['정의와 인권 그리고 민주주의를 갈망하는 국민 여러분!',
 '더불어민주당 원내대표 안양 만안구 출신의 국회의원 이종걸입니다.',
 '국민의 사생활을 보호하고 인권과 민주주의를 지켜내기 위한 무제한 토론이 185시간에 이르렀습니다. 2월 23일 오후 7시 5분에 시작했습니다.',
 '우리 당의 김광진 의원님을 시작으로 역사적인 장정이 시작되었습니다. 문병호 의원님, 은수미 의원님, 이름을 불러도 눈물이 납니다. 박원석 의원님, 유승희 의원님, 최민희 의원님 그리고 정의당의 김제남 의원님, 우리 당의 신경민 의원님, 마음에 상처를 많이 받은 우리 강기정 의원님, 김경협 의원님, 정의당의 서기호 의원님, 우리 당의 김현 의원님, 김용익 의원님, 배재정 의원님, 전순옥 의원님, 추미애 의원님, 정청래 의원님, 진선미 의원님, 최규성 의원님, 박혜자 의원님, 오제세 의원님, 권은희 의원님, 이학영 의원님, 홍종학 의원님, 서영교 의원님, 최원식 의원님, 홍익표 의원님, 이언주 의원님, 전정희 의원님, 임수경 의원님, 안민석 의원님, 김기준 의원님, 김관영 의원님, 박영선 의원님, 주승용 의원님 그리고 정의당의 정진후 의원님, 심상정 의원님이 열정적으로 토론에 참여하셨습니다.',
 '이 열정으로 국민들은 국회가, 국회에서 국회의원들이 어떻게 활동하고 있는지를 일면이나마 아시게 된 것 같습니다. 저희조차 몰랐습니다. 그것이 국민과 동떨어져 있었다는 증거를 스스로 알고 스스로 인식하고 자책했습니다.',
 '선거철입니다. 선거구민을 만나면 먼저 하는 말이 싸우지 말라고 합니다. 잘 서로 논의하고 타협해서 싸우지 말고 잘 하라고 합니다.',
 '연일 많은 언론보도를 통해서 본회의장에서 무슨 검색하다가 걸려서 웃음거리가 되는 사건들을 국민들은 접했습니다. 또 무슨 사건 사고에 휘말리는 국회의원들을 큰 글자로 접했습니다.',
 '저는 국민들이 평가하는 국회의원들에 대한 생각은 그래도 뭔가 있으면서, 언론에서 접하는 그런 사건들을 떠올리면서 잘 하라는 격려로 어떤 레토릭으로 얘기

앞서 전처리한 방법과 같은 방식으로 각 의원들의 발언을 각각 전처리하여 명사들의 리스트로 만든다.

In [13]:
%%time

by_speaker_normalized_path = 'intermediate/by_speaker_normalized/'

for name in names:
    speaker_text = '\n'.join(by_name[name])
    speaker_normalized_text = normalize(speaker_text)

    with open(by_speaker_normalized_path + name + '.txt', 'w') as f:
        f.write(' '.join(speaker_normalized_text))

CPU times: user 1min 45s, sys: 1.42 s, total: 1min 47s
Wall time: 1min 48s


## 분석 대상 단어 선정하기

의원 별 발언을 분석할 때 필리버스터와 어느 정도 관련된 단어만을 대상으로 하기 위해 분석 대상 단어를 선정한다. 전체 필리버스터 내용 중 일정 수 이상 출현한 단어만을 분석 대상 단어로 정하였다.

In [14]:
from collections import Counter

full_counter = Counter(normalized_text)

target_words = [key for key, value in full_counter.most_common(500)]

분석 대상 단어 500개를 출현 빈도 순으로 나열해보자.

In [47]:
print(target_words)

['테러', '국정원', '국민', '정보', '국가', '우리', '방지법', '대한', '지금', '사건', '대통령', '의원', '국회', '사람', '때문', '정부', '기관', '가지', '그것', '문제', '내용', '정치', '사실', '활동', '경우', '법안', '미국', '권한', '국가정보원', '수사', '감청', '행위', '인권', '규정', '대한민국', '정보기관', '모든', '위원회', '여러분', '침해', '비상', '수집', '상황', '사태', '법률', '관련', '헌법', '북한', '대책', '안전', '사회', '얘기', '자유', '감시', '방지', '당시', '다른', '위험', '통제', '새누리당', '통신', '하나', '다시', '안보', '위해', '조직', '통과', '이유', '권력', '조사', '업무', '조항', '필리버스터', '자료', '기구', '조치', '야당', '보고', '의장', '부분', '나라', '말씀', '생각', '국제', '회의', '시간', '한국', '직권상정', '정권', '직원', '민주주의', '중앙', '개입', '이후', '저희', '간첩', '개인', '바로', '범죄', '우리나라', '금융', '영장', '이제', '과정', '단체', '관계', '대응', '제정', '토론', '여기', '다음', '보호', '사찰', '부의', '여러', '계속', '정도', '목적', '국내', '또한', '인물', '제대로', '권리', '여당', '이지', '고문', '결과', '경찰', '정말', '현재', '박근혜', '기본권', '발언', '기능', '개혁', '일이', '가장', '절차', '제도', '모두', '언론', '방법', '위협', '민주', '과거', '선거', '센터', '예산', '조작', '국회의원', '대해', '지난', '해외', '장관', '제한', '의견', '그때', '불법', '사항', '대상', '국회의장', '무엇

대부분 필리버스터 전체 텍스트의 워드 클라우드에서 보았던 단어들이라는 것을 알 수 있다.

##  TF-IDF로 의원 별 발언 특징 잡아내기

TF-IDF(Term Frequency - Inverse Document Frequency) 여러 문서 중에서 특정 문서가 특징적으로 갖고 있는 키워드를 추출하는 기법이다. 어떤 단어의 특정 문서에 대한 TF-IDF 점수는 그 단어가 그 문서에서 자주 쓰일 수록 높아지고, 다른 문서들에서도 자주 쓰일 수록 낮아진다. TF-IDF 점수는 TF 점수와 IDF 점수의 곱으로 나타낼 수 있다.

$$ {\mathrm  {tfidf}}(t,d,D)={\mathrm  {tf}}(t,d)\times {\mathrm  {idf}}(t,D) $$

여기서 t는 특정 단어, d는 특정 문서, D는 전체 문서 집합이다.

TF와 IDF를 구하는 방법은 여러가지가 있다. 이 분석에서는 다음 공식을 사용했다.

$$ \mathrm {tf} (t,d) = f_{t,d}$$

$$  \mathrm{idf}(t, D) =  \log \frac{N}{|\{d \in D: t \in d\}|} $$

- $f_{t,d}$는 단어 t가 문서 d에서 출현한 빈도
- N은 문서의 갯수 $N = {|D|}$

이 분석에서 문서 하나는 한 의원의 발언 내용이고, 전체 문서 집합은 38명 의원들의 필리버스터 내용 집합이 된다. TF-IDF를 이용하면 '테러', '국정원' 등 모든 의원들이 공통적으로 언급한 단어들은 제외하고, 어느 한 의원만 자주 언급한 키워드를 볼 수 있다. 이는 그 의원이 주로 든 테러방지법 반대 근거가 될 때도 있고, 그 의원의 발언 스타일이나 언어 습관 등도 반영된다. 이제 의원 한명 한명의 차별화된 발언 특징들을 잡아내고 그 키워드들로만 워드 클라우드를 그려보자.

In [16]:
counters = []
for name in names:
    with open(by_speaker_normalized_path + name + '.txt') as f:
        speaker_text = f.read()        
    counter = Counter(speaker_text.split())
    counters.append(counter)

In [17]:
import math

def tf(word, counter):
    return counter[word]

def idf(word, corpus):
    num_containing = sum(1 for counter in corpus if counter[word] > 0)
    return math.log(len(corpus) / (1 + num_containing))

def tfidf(word, counter, corpus):
    """TF-IDF 점수를 구한다."""
    return tf(word, counter) * idf(word, corpus)

In [18]:
def tfidf_scores(name):
    """의원 한 명에 대해서 분석 대상 단어들의 TF-IDF 점수를 계산한다."""
    scores = {}
    with open(by_speaker_normalized_path + name + '.txt') as f:
        speaker_text = f.read()        
    counter = Counter(speaker_text.split())

    for word in target_words:
        score = tfidf(word, counter, counters)
        scores[word] = score
    scores_sorted = sorted(scores.items(), key=lambda x:x[1], reverse=True)
    return scores_sorted

In [30]:
def tfidf_cloud(name):
    """의원 한 명에 대한 분석 대상 단어들의 TF-IDF 점수를 워드 클라우드로 그린다."""
    scores = tfidf_scores(name)
    tfidf_wc = WC.generate_from_frequencies(scores)
    tfidf_wc.to_file('figures/' + name + '.png')

In [34]:
for name in names:
    tfidf_cloud(name)

예를 들어, 은수미 의원의 IF-IDF가 높은 단어들을 살펴보면 다음과 같다.

In [25]:
tfidf_scores('은수미')[:10]

[('무죄', 15.910992724611031),
 ('고문', 15.460363280321191),
 ('해킹', 14.656104239982282),
 ('시설', 11.909884332496093),
 ('관여', 11.604502682944911),
 ('테러리즘', 9.291243008257188),
 ('후보', 8.34877167750788),
 ('기업', 8.24530453788191),
 ('기존', 7.618244306034902),
 ('도대체', 7.027548573237704)]

이 점수들을 바탕으로 워드 클라우드를 그려보자.

### 1. 은수미 의원

In [43]:
tfidf_cloud('은수미')

![은수미](figures/은수미.png)

**무죄**, **고문**과 같은 키워드가 두드러진다. 실제로 은수미 의원은 과거 정권이 자행한 고문의 사례들과 국정원이 간첩으로 몰았지만 무죄로 판결났던 사례들을 나열하며 국정원의 부당함을 강변했다.

> "대표적 예를 좀 들어 보면요, 1975년 2차 인혁당 사건으로 구속되어서 사형당한―무죄 판결을 받았지요―어쨌든 사형당한 이수병 등의 시신에는 전기고문의 흔적이 완연히 남아 있었으며 가족들은 그 시신조차 받지 못하였습니다."

> "고문은 제가 아까도 말씀드렸듯이 신체적 고문과 정신적 고문이 있습니다. 정신적 고문이 끼치는 영향은 엄청납니다. 위협, 협박 이런 것도 고문이 실제로 됩니다. 온갖 방법의 정신적 고문이 있습니다. 그런데 이런 고문조차도 가능한 대규모 어떤 권력기관을 만들려는 게 아닌가?"

> "예를 들어 폰 감청, 구속, 시위 시의 폭압, 국정원의 의도, 은행 개인계좌 등등 등등. 국정원의 조작으로 간첩사건이 무죄로 판결되었는데 판결되기까지 당사자는 엄청난 고통을 받았습니다. 심지어는 아이와 친척들까지 고통을 받았습니다. 과거에도 그러했는데 이번 테러법은 평범한 국민을 이처럼 내몰 수 있는 그러한 사례를 더 많이 만들게 하자는 법 아닐까요?"

> *은수미 의원 필리버스터 중*

참고로, 은수미 의원은 본인이 노동운동을 하다가 안기부에 잡혀 고문을 당한 경험이 있다. 고문후유증으로 유산까지 했다고 한다.

### 2. 박영선 의원

In [41]:
tfidf_cloud('박영선')

![박영선](figures/박영선.png)

박영선 의원의 워드 클라우드에서는 **의석**이란 키워드가 가장 눈에 띈다. 실제로 박영선 의원은 필리버스터 발언 도중 야당의 의석수가 적음을 강조하며 선거에서 야당에게 과반 의석을 달라는 발언을 자주 하였다.

> "과반 의석을 갖지 못한 소수 야당으로서는 정말 이 거대 여당, 공룡 새누리당의 횡포, 박근혜 정권의 폭주, 이것을 막아 내는 데는 역부족이었다는 생각이 듭니다."

> "과반 의석을 가져야 국민 여러분의 그 마음을 행동으로, 실천으로 옮길 수 있습니다. "

> "국민 여러분께서 분노하신 만큼 4월 13일 총선에서 야당을 찍어 주십시오. 야당에게 과반 의석을 주셔야 여러분들이 원하던 그런 평화롭고 행복한 대한민국을 만들 수 있습니다."

> *박영선 의원 필리버스터 중*

이 발언으로 [필리버스터는 선거운동이라는 논란](http://www.sisaweek.com/news/articleView.html?idxno=64840)을 자초하기도 하였다.

### 3. 이종걸 의원

In [56]:
tfidf_cloud('이종걸')

![이종걸](figures/이종걸.png)

이종걸 의원의 워드 클라우드에는 특이하게도 **빙자**라는 키워드가 보인다. 이는 이종걸 의원이 의도적으로 '테러방지법'을 '테러빙자법'으로 불렀기 때문이다. 이종걸 의원은 필리버스터 처음부터 끝까지 '테러빙자법'이라고 부르길 유지하며 결국 총 123번이나 '테러빙자법'이란 말을 언급했다.

> "우리 당이 무제한 토론을 하고 있는 테러방지법의 정식 명칭은 ‘국민보호와 공공안전을 위한 테러방지법’입니다. 그러나 우리 당은 그 법을 ‘국민감시와 공공사찰을 위한 테러방지빙자법’ 또는 ‘국정원 무제한 감청 및 금융정보 취득법’이라고 읽겠습니다. 저는 이제부터 이 법을 줄여 가지고 ‘테러빙자법’이라고 하겠습니다."

> "테러빙자법이 없어서 통영함의 음파탐지기가 70년대인 것일까요? 테러빙자법이 없어서 어뢰 36발을 실험했더니 30발이 불발이었나요? 테러빙자법이 없어서 국정원은 가짜 간첩을 조작해서 만들어 냈나요? 테러빙자법이 없어서 국정원은 김정은을 김정운이라고 알고 있었나요?"

> *이종걸 의원 필리버스터 중*

다른 의원들에 대한 워드 클라우드는 [여기](https://github.com/dreamgonfly/filibuster/tree/master/figures)에서 볼 수 있다.

## Conclusion

이 노트북에서는 필리버스터 전체의 키워드와 각 의원 별로 특징적인 키워드를 워드 클라우드로 나타내 보았다. 전체 워드 클라우드에서는 테러방지법의 주요 반대 이유가 국정원에 대한 불신이며, 국가가 국민의 정보를 가져가는 것을 우려하기 때문이라는 점을 알 수 있었다. 또한 TF-IDF 기법을 이용해서 은수미 의원의 고문과 무죄, 박영선 의원의 의석, 이종걸 의원의 테러'빙자'법과 같이 각 의원 별로 키워드를 뽑아냈다. 

2016년 테러방지법 반대 필리버스터는 세계에서 가장 긴 필리버스터이며 한국 정치사에서 중요한 사건이다. 하지만 너무 길기 때문에 필리버스터 내용을 처음부터 끝까지 읽기는 매우 힘들다. 이와 같은 긴 이야기를 데이터로 핵심어를 추출하여 요약하여 전달할 수 있다는 데 이번 워드 클라우드 시각화는 의미가 있었다.

## Potential future work

시간이 있다면 할 수 있는 일들은 아직 많이 남아있다.

- 사전을 구축하면 국가정보원-국정원, 국가정보원장-국정원장 같은 동의어를 적절히 처리할 수 있고, '테러방지법'을 '테러'와 '방지법'으로 나누는 대신 한 단어로 처리할 수 있다.

- 명사 뿐 아니라 동사를 포함해서 키워드에 대해 어떻게 서술했는지 알아보기

- 워드 클라우드 말고도 단어들 사이의 관계를 나타낼 수 있는 네트워크 시각화를 시도하기

- 의원들 사이의 거리를 정의해서 특이한 발언을 한 의원들을 자동으로 탐지해보는 것도 재미있을 것이다. 

---

##### dreamgonfly@gmail.com